## Interview Success Prediction

Given *data about resumes*, let's try to predict whether a candidate will **pass their interview** based on their resume.

We will use three models to make our predictions, and PCA for dimensionality reduction to make our predictions.

Data source: https://www.kaggle.com/datasets/vingkan/strategeion-resume-skills

### Importing Libraries

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [4]:
data = pd.read_csv('archive/resumes_development.csv')
data

,Unnamed: 0,Interview,Adobe Illustrator,Adobe Photoshop,Agile Methodologies,Analytical Skills,Art,Art Direction,Auditing,AutoCAD,Automation,Automotive,Baseball,Basketball,Branding,Budgeting,Business Analysis,Business Intelligence,Business Planning,Business Process,Business Process Improvement,Business Services,Business Strategy,Change Management,Coaching,Communication,Company Research,Construction,Contract Negotiation,Contractual Agreements,Customer Relationship Management,Customer Satisfaction,Customer Service,Data Analysis,Databases,Decision Making,Department of Defense,Design,Development Tools,Digital Marketing,Digital Media,Diving,Drawing,Ecommerce,Editing,Electronics,Email,Employee Benefits Design,Energy,Enterprise Software,Entrepreneurship,Event Planning,Facilitation,Facility Management,Finance,Financial Analysis,Financial Reporting,Financial Risk,Food and Beverage,Football,Forecasting,Fundraising,Gas,General Ledger,Golf,Government Agencies,Grant Writing,Graphic Design,Graphics,Grocery,Gymnastics,HTML,HTML5,Healthcare,Healthcare Management,Heavy Equipment,Helping Clients Succeed,Higher Education,History,Hospitality Industry,Hospitals,Information Systems,Information Technology,Insurance,Integrated Marketing,Integration,Internal Audit,Interpersonal Skills,Interviewing,Investments,Invoicing,JQuery,Java,Javascript,Job Description Development,Job Scanning,Joint Ventures,Journalism,Journals,K12 Education,Kanban,Key Account Development,Key Performance Indicators,Kindergarten,Knowledge Base,Knowledge Management,Knowledge Sharing,Laboratory Skills,Leadership,Leadership Development,Lean Manufacturing,Legal Issues,Legal Writing,Linux,Litigation,Local Marketing,Logistics Management,Marine Corps,Mechanical Engineering,Medical Compliance,Military Aviation,Military Weapons,Negotiation,Network Administration,Network Attached Storage,Network Communications,Network Systems,New Business Opportunities,New Hires,Newsletters,Nonprofit Organizations,Nutrition,Office Administration,Online Advertising,Operating Systems,Operations Management,Oracle Database,Order Fulfillment,Organizational Development,Organizational Structure,Outsourcing,Partnerships,Performance Tuning,Presentations,Pricing Strategy,Problem Solving,Program Management,Project Planning,Public Policy,Public Relations,Public Speaking,Qualifying Prospects,Qualitative Research,Quality Auditing,Quality Improvement,Quality System,Quantitative Research,Query Writing,Queues,QuickBooks,Quotations,Range,Real Estate,Records,Recruiting,Regulations,Requirements Analysis,Research,Residential Homes,Retail,Reviews,SQL,Sales Management,Schedules,Soccer,Social Media,Software,Software Documentation,Staff Development,Strategic Planning,Strategy,Supervisory Skills,Swimming,Teaching,Team Building,Team Leadership,Team Management,Teamwork,Telecommunications,Testing,Time Management,Training,Troubleshooting,Underwriting,Unified Modeling Language,University Teaching,Unix,User Acceptance Testing,User Experience,User Interface Design,Validation,Vendor Management,Vendors,Video,Video Editing,Video Production,Visual Merchandising,Volleyball,Volunteer Management,Volunteering,Water Resource Management,Web Applications,Web Design,Web Development,Windows,Windows Server,Wireless Technologies,WordPress,Workshops,Writing,Veteran,Female,URM,Disability
0,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619 entries, 0 to 618
Columns: 224 entries, Unnamed: 0 to Disability
dtypes: float64(219), int64(5)
memory usage: 1.1 MB


### Preprocessing

In [6]:
df = data.copy()

In [7]:
# Drop the index column
df = df.drop('Unnamed: 0', axis=1)

In [8]:
df

,Interview,Adobe Illustrator,Adobe Photoshop,Agile Methodologies,Analytical Skills,Art,Art Direction,Auditing,AutoCAD,Automation,Automotive,Baseball,Basketball,Branding,Budgeting,Business Analysis,Business Intelligence,Business Planning,Business Process,Business Process Improvement,Business Services,Business Strategy,Change Management,Coaching,Communication,Company Research,Construction,Contract Negotiation,Contractual Agreements,Customer Relationship Management,Customer Satisfaction,Customer Service,Data Analysis,Databases,Decision Making,Department of Defense,Design,Development Tools,Digital Marketing,Digital Media,Diving,Drawing,Ecommerce,Editing,Electronics,Email,Employee Benefits Design,Energy,Enterprise Software,Entrepreneurship,Event Planning,Facilitation,Facility Management,Finance,Financial Analysis,Financial Reporting,Financial Risk,Food and Beverage,Football,Forecasting,Fundraising,Gas,General Ledger,Golf,Government Agencies,Grant Writing,Graphic Design,Graphics,Grocery,Gymnastics,HTML,HTML5,Healthcare,Healthcare Management,Heavy Equipment,Helping Clients Succeed,Higher Education,History,Hospitality Industry,Hospitals,Information Systems,Information Technology,Insurance,Integrated Marketing,Integration,Internal Audit,Interpersonal Skills,Interviewing,Investments,Invoicing,JQuery,Java,Javascript,Job Description Development,Job Scanning,Joint Ventures,Journalism,Journals,K12 Education,Kanban,Key Account Development,Key Performance Indicators,Kindergarten,Knowledge Base,Knowledge Management,Knowledge Sharing,Laboratory Skills,Leadership,Leadership Development,Lean Manufacturing,Legal Issues,Legal Writing,Linux,Litigation,Local Marketing,Logistics Management,Marine Corps,Mechanical Engineering,Medical Compliance,Military Aviation,Military Weapons,Negotiation,Network Administration,Network Attached Storage,Network Communications,Network Systems,New Business Opportunities,New Hires,Newsletters,Nonprofit Organizations,Nutrition,Office Administration,Online Advertising,Operating Systems,Operations Management,Oracle Database,Order Fulfillment,Organizational Development,Organizational Structure,Outsourcing,Partnerships,Performance Tuning,Presentations,Pricing Strategy,Problem Solving,Program Management,Project Planning,Public Policy,Public Relations,Public Speaking,Qualifying Prospects,Qualitative Research,Quality Auditing,Quality Improvement,Quality System,Quantitative Research,Query Writing,Queues,QuickBooks,Quotations,Range,Real Estate,Records,Recruiting,Regulations,Requirements Analysis,Research,Residential Homes,Retail,Reviews,SQL,Sales Management,Schedules,Soccer,Social Media,Software,Software Documentation,Staff Development,Strategic Planning,Strategy,Supervisory Skills,Swimming,Teaching,Team Building,Team Leadership,Team Management,Teamwork,Telecommunications,Testing,Time Management,Training,Troubleshooting,Underwriting,Unified Modeling Language,University Teaching,Unix,User Acceptance Testing,User Experience,User Interface Design,Validation,Vendor Management,Vendors,Video,Video Editing,Video Production,Visual Merchandising,Volleyball,Volunteer Management,Volunteering,Water Resource Management,Web Applications,Web Design,Web Development,Windows,Windows Server,Wireless Technologies,WordPress,Workshops,Writing,Veteran,Female,URM,Disability
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0

In [9]:
# Split df into X and y
y = df['Interview'].copy()
X = df.drop('Interview', axis=1).copy()

In [10]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)

In [11]:
X_train

,Adobe Illustrator,Adobe Photoshop,Agile Methodologies,Analytical Skills,Art,Art Direction,Auditing,AutoCAD,Automation,Automotive,Baseball,Basketball,Branding,Budgeting,Business Analysis,Business Intelligence,Business Planning,Business Process,Business Process Improvement,Business Services,Business Strategy,Change Management,Coaching,Communication,Company Research,Construction,Contract Negotiation,Contractual Agreements,Customer Relationship Management,Customer Satisfaction,Customer Service,Data Analysis,Databases,Decision Making,Department of Defense,Design,Development Tools,Digital Marketing,Digital Media,Diving,Drawing,Ecommerce,Editing,Electronics,Email,Employee Benefits Design,Energy,Enterprise Software,Entrepreneurship,Event Planning,Facilitation,Facility Management,Finance,Financial Analysis,Financial Reporting,Financial Risk,Food and Beverage,Football,Forecasting,Fundraising,Gas,General Ledger,Golf,Government Agencies,Grant Writing,Graphic Design,Graphics,Grocery,Gymnastics,HTML,HTML5,Healthcare,Healthcare Management,Heavy Equipment,Helping Clients Succeed,Higher Education,History,Hospitality Industry,Hospitals,Information Systems,Information Technology,Insurance,Integrated Marketing,Integration,Internal Audit,Interpersonal Skills,Interviewing,Investments,Invoicing,JQuery,Java,Javascript,Job Description Development,Job Scanning,Joint Ventures,Journalism,Journals,K12 Education,Kanban,Key Account Development,Key Performance Indicators,Kindergarten,Knowledge Base,Knowledge Management,Knowledge Sharing,Laboratory Skills,Leadership,Leadership Development,Lean Manufacturing,Legal Issues,Legal Writing,Linux,Litigation,Local Marketing,Logistics Management,Marine Corps,Mechanical Engineering,Medical Compliance,Military Aviation,Military Weapons,Negotiation,Network Administration,Network Attached Storage,Network Communications,Network Systems,New Business Opportunities,New Hires,Newsletters,Nonprofit Organizations,Nutrition,Office Administration,Online Advertising,Operating Systems,Operations Management,Oracle Database,Order Fulfillment,Organizational Development,Organizational Structure,Outsourcing,Partnerships,Performance Tuning,Presentations,Pricing Strategy,Problem Solving,Program Management,Project Planning,Public Policy,Public Relations,Public Speaking,Qualifying Prospects,Qualitative Research,Quality Auditing,Quality Improvement,Quality System,Quantitative Research,Query Writing,Queues,QuickBooks,Quotations,Range,Real Estate,Records,Recruiting,Regulations,Requirements Analysis,Research,Residential Homes,Retail,Reviews,SQL,Sales Management,Schedules,Soccer,Social Media,Software,Software Documentation,Staff Development,Strategic Planning,Strategy,Supervisory Skills,Swimming,Teaching,Team Building,Team Leadership,Team Management,Teamwork,Telecommunications,Testing,Time Management,Training,Troubleshooting,Underwriting,Unified Modeling Language,University Teaching,Unix,User Acceptance Testing,User Experience,User Interface Design,Validation,Vendor Management,Vendors,Video,Video Editing,Video Production,Visual Merchandising,Volleyball,Volunteer Management,Volunteering,Water Resource Management,Web Applications,Web Design,Web Development,Windows,Windows Server,Wireless Technologies,WordPress,Workshops,Writing,Veteran,Female,URM,Disability
180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [13]:
y_train.value_counts()

Interview
0.0    285
1.0    148
Name: count, dtype: int64

In [14]:
# Scale X
scaler = StandardScaler()

scaler.fit(X_train)

X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

### Training/Results

In [15]:
models = {
    "Logistic Regression": LogisticRegression(),
    "      Decision Tree": DecisionTreeClassifier(),
    "      Random Forest": RandomForestClassifier()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

Logistic Regression trained.
      Decision Tree trained.
      Random Forest trained.


In [16]:
for name, model in models.items():
    print(name + " Accuracy: {:.2f}%".format(model.score(X_test, y_test)*100))

Logistic Regression Accuracy: 96.24%
      Decision Tree Accuracy: 86.02%
      Random Forest Accuracy: 88.71%


### Training/Results With Dimensionality Reduction

In [48]:
n_components = 5

pca = PCA(n_components=n_components)
pca.fit(X_train)

X_train_reduced = pd.DataFrame(pca.transform(X_train), index=X_train.index, columns=["PC" + str(i) for i in range(1, n_components + 1)])
X_test_reduced = pd.DataFrame(pca.transform(X_test), index=X_test.index, columns=["PC" + str(i) for i in range(1, n_components + 1)])

In [49]:
X_train_reduced

,PC1,PC2,PC3,PC4,PC5
180,0.250793,-0.124216,-0.016524,-0.473988,-0.412487
403,-1.311471,-0.750515,1.089016,-1.516481,-0.215803
42,-1.920149,-0.295278,-0.151425,-1.190229,-0.399183
323,-1.961690,0.948809,-2.544759,0.951037,-1.984682
428,-2.507523,-1.567815,-2.272444,2.315534,2.789380
...,...,...,...,...,...
129,-1.702744,-1.895603,0.774699,2.975062,1.031635
144,1.662989,-1.621784,1.246217,-0.070243,-0.639196
72,1.521258,0.256343,0.138130,-1.850636,1.285443
235,4.341801,-1.069833,-0.633723,0.569538,-0.605878


In [50]:
pca.explained_variance_ratio_

array([0.02120111, 0.01361142, 0.01289543, 0.01265714, 0.01247436])

In [51]:
models = {
    "Logistic Regression": LogisticRegression(),
    "      Decision Tree": DecisionTreeClassifier(),
    "      Random Forest": RandomForestClassifier()
}

for name, model in models.items():
    model.fit(X_train_reduced, y_train)
    print(name + " trained.")

Logistic Regression trained.
      Decision Tree trained.
      Random Forest trained.


In [52]:
for name, model in models.items():
    print(name + " Accuracy: {:.2f}%".format(model.score(X_test_reduced, y_test)*100))

Logistic Regression Accuracy: 95.16%
      Decision Tree Accuracy: 94.62%
      Random Forest Accuracy: 95.16%
